* Pseudo labeling
    * Jeremy made a batch iterator class that combines training data with test data that you labeled with a previous model.
    * You can also add validation data, at the end of training to improve the last bit.
* Embeddings
    * embeddings() just looks up the embeddings of a certain called user/movie index. Then you merge() them with a dot product, add biases and compare to the actual rating to find the error.
* RNN's
    * It's really all about memory.
    * 
    